In [18]:
import torch
resnet34_model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_resnet32", pretrained=True)

Using cache found in C:\Users\nick/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


In [11]:
import os
import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10

from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
import torchvision.models as models

import tarfile


In [12]:
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
AVAIL_GPUS


1

In [7]:
# Dowload the dataset
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar100.tgz"
download_url(dataset_url, '.')

169168896it [00:28, 5920949.21it/s]                                


In [9]:
with tarfile.open('./cifar100.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

In [8]:
data_dir = './data/cifar100'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['test', 'train']
['aquatic_mammals', 'fish', 'flowers', 'food_containers', 'fruit_and_vegetables', 'household_electrical_devices', 'household_furniture', 'insects', 'large_carnivores', 'large_man-made_outdoor_things', 'large_natural_outdoor_scenes', 'large_omnivores_and_herbivores', 'medium_mammals', 'non-insect_invertebrates', 'people', 'reptiles', 'small_mammals', 'trees', 'vehicles_1', 'vehicles_2']


In [9]:
# Data transforms (normalization & data augmentation)
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = transforms.Compose([transforms.RandomCrop(32, padding=4, padding_mode='reflect'), 
                         transforms.RandomHorizontalFlip(), 
                         # tt.RandomRotate
                         # tt.RandomResizedCrop(256, scale=(0.5,0.9), ratio=(1, 1)), 
                         # tt.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                         transforms.ToTensor(), 
                         transforms.Normalize(*stats,inplace=True)])
valid_tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(*stats)])

In [10]:
# PyTorch datasets
train_data = ImageFolder(data_dir+'/train', train_tfms)
valid_data = ImageFolder(data_dir+'/test', valid_tfms)

In [13]:
# init the model

# init Dataloader from MNIST Dataloader

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)

# initialize the trainer
trainer = Trainer(
    gpus = AVAIL_GPUS,
    max_epochs = 4,
    progress_bar_refresh_rate = 20,
)

C:\Users\nick\.conda\envs\cv\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [1]:
import os
import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10

from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
import torchvision.models as models

import tarfile


In [2]:
class CifarModel(LightningModule):
    def __init__(self):
        super().__init__()

        # init a pretrained resnet
        backbone = models.resnet34(pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)


        self.PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
        AVAIL_GPUS = min(1, torch.cuda.device_count())
        self.BATCH_SIZE = 256 if AVAIL_GPUS else 64
        print(f"If GPU's are available - {AVAIL_GPUS}")


        # Data transforms (normalization & data augmentation)
        stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        self.train_tfms = transforms.Compose([transforms.RandomCrop(32, padding=4, padding_mode='reflect'), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize(*stats,inplace=True)])
        self.valid_tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(*stats)])

        # use the pretrained model to classify cifar-10 (10 image classes)
        num_target_classes = 100
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return loss

    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     logits = self(x)
    #     loss = F.cross_entropy(logits, y)
    #     preds = torch.argmax(logits, dim=1)
    #     self.accuracy(preds, y)

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.accuracy(preds, y)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.02)

    ####################
    # DATA RELATED HOOKS
    ####################

    def prepare_data(self):
        # download
        if not os.path.exists('data/'):
            print("Data doesn't exist on memory downloading it now")
            dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar100.tgz"
            download_url(dataset_url, '.')

            with tarfile.open('./cifar100.tgz', 'r:gz') as tar:
                tar.extractall(path='./data')

    def setup(self, stage=None):
        data_dir = './data/cifar100'


        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:

            # PyTorch datasets
            self.train_data = ImageFolder(data_dir+'/train', self.train_tfms)

            # self.cifar_train, self.cifar_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.valid_data = ImageFolder(data_dir+'/test', self.valid_tfms)

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=self.BATCH_SIZE, 
            num_workers= 1 #os.cpu_count()
            )

    def test_dataloader(self):
        return DataLoader(
            self.valid_data,
            batch_size=self.BATCH_SIZE, 
            num_workers=1 #os.cpu_count()
            )


In [3]:
cifar_model = CifarModel()
from pytorch_lightning.callbacks import ModelSummary

AVAIL_GPUS = min(1, torch.cuda.device_count())


trainer = Trainer(
    callbacks=[ModelSummary(max_depth=-1)],
    precision = 16,
    gpus = AVAIL_GPUS,
    max_epochs = 3,
    auto_scale_batch_size="binsearch",
    accumulate_grad_batches=5,
    progress_bar_refresh_rate=10
)
trainer.fit(cifar_model)



Using 16bit native Automatic Mixed Precision (AMP)
C:\Users\nick\.conda\envs\cv\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


If GPU's are available - 1


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

    | Name                               | Type              | Params
---------------------------------------------------------------------------
0   | feature_extractor                  | Sequential        | 21.3 M
1   | feature_extractor.0                | Conv2d            | 9.4 K 
2   | feature_extractor.1                | BatchNorm2d       | 128   
3   | feature_extractor.2                | ReLU              | 0     
4   | feature_extractor.3                | MaxPool2d         | 0     
5   | feature_extractor.4                | Sequential        | 221 K 
6   | feature_extractor.4.0              | BasicBlock        | 74.0 K
7   | feature_extractor.4.0.conv1        | Conv2d            | 36.9 K
8   | feature_extractor.4.0.bn1          | BatchNorm2d       | 128   
9   | feature_extractor.4.0.relu         | ReLU              | 0     
10  | feature_extractor.4.0.conv2        | Conv2d            | 36.9 K
11  | feature_extractor.4.0.bn2          

Epoch 2: 100%|██████████| 196/196 [00:38<00:00,  5.03it/s, loss=3.91, v_num=7]


In [29]:
import torchvision.models as models

class LitModel(LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes
        
        # transfer learning if pretrained=True
        self.feature_extractor = models.resnet34(pretrained=True)
        # layers are frozen by using eval()
        self.feature_extractor.eval()
        # freeze params
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
        
        n_sizes = self._get_conv_output(input_shape)

        self.classifier = nn.Linear(n_sizes, num_classes)
  
    # returns the size of the output tensor going into the Linear layer from the conv block.
    def _get_conv_output(self, shape):
        batch_size = 1
        tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(tmp_input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size
        
    # returns the feature tensor from the conv block
    def _forward_features(self, x):
        x = self.feature_extractor(x)
        return x
    
    # will be used during inference
    def forward(self, x):
       x = self._forward_features(x)
       x = x.view(x.size(0), -1)
       x = F.log_softmax(self.classifier(x), dim=1)
       
       return x

In [ ]:
LitModel

In [ ]:
class LitMNIST(LightningModule):
    def __init__(self, data_dir=PATH_DATASETS, hidden_size=16, learning_rate=2e-4):

        super().__init__()

        # Set our init args as class attributes
        self.data_dir = data_dir
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

        # Hardcode some dataset specific attributes
        self.num_classes = 10
        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
        )

        # Define PyTorch model
        self.model = nn.Sequential(
            nn.Conv2d(1, hidden_size, (3, 3)),
            nn.ReLU(),
            nn.Conv2d(hidden_size, hidden_size, (3, 3)),
            nn.ReLU(),
            nn.Conv2d(hidden_size, hidden_size, (3, 3), 2),
            nn.ReLU(),
            nn.Conv2d(hidden_size, self.num_classes, (3, 3)),
            nn.ReLU(),
            nn.AvgPool2d(9),
        )

        self.accuracy = Accuracy()

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    ####################
    # DATA RELATED HOOKS
    ####################

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=BATCH_SIZE, num_workers=os.cpu_count())

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=BATCH_SIZE, num_workers=os.cpu_count())

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=BATCH_SIZE, num_workers=os.cpu_count())

In [9]:
if not os.path.exists('data/'):
    print(True)
else:
    print('ac')

ac
